In [1]:
import pandas as pd


df_buscas = pd.read_excel("buscas.xlsx")
display(df_buscas)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [213]:
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from time import sleep


# ler base de dados
df_buscas = pd.read_excel("buscas.xlsx")


# armazenar caminho dos sites de pesquisa
caminhos = ["https://shopping.google.com.br/?pli=1", "https://www.buscape.com.br/?og=19220&og=19220&gclid=Cj0KCQjwiZqhBhCJARIsACHHEH_J8GkuCKXw8zsWebQLekAbLNQvl2k2uQRMtR-KTUyu5MFUXUY7PL8aAr6sEALw_wcB"]

# abrir navegador
navegador = webdriver.Chrome()




df_atualizada = pd.DataFrame()
lista_final = []


# pesquisar na barra de busca cada produto
for busca in df_buscas.index:
    
    # pegar primeiro elemento (google)
    navegador.get(caminhos[0])

    # esperar página carregar
    elemento = WebDriverWait(navegador, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/c-wiz[3]/div/div/div[1]/div/div/div')))
    
    produto = df_buscas.loc[busca, "Nome"]
    banido = df_buscas.loc[busca, "Termos banidos"]
    pmin = df_buscas.loc[busca, "Preço mínimo"]
    pmax = df_buscas.loc[busca, "Preço máximo"]
    
    # buscar produto eliminando termo banido (-termo_que não quero)
    navegador.find_element(By.ID, "REsRA").send_keys(f"{produto} -{banido}", Keys.ENTER)
    
    # aguardar
    elemento = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'sh-dr__restricts')))
    
    # filtrar por preços
    navegador.find_element(By.NAME, "lower").send_keys(f"{pmin}")
    navegador.find_element(By.NAME, "upper").send_keys(f"{pmax}", Keys.ENTER)
    
    # aparecer do menor para o maior preço
    navegador.find_element(By.CLASS_NAME, "Yf5aUd").click()
    sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="lb"]/div/g-menu/g-menu-item[3]').click()
    
    #lista_produtos = navegador.find_elements(By.CLASS_NAME, "KZmu8e C4IbMe")
    #lista_produtos.append(navegador.find_elements(By.CLASS_NAME, "KZmu8e Ehwxtb"))
    #print(lista_produtos)
    
    

    # guardar produtos em uma lista
    #lista_produtos = navegador.find_elements(By.CLASS_NAME, "ljqwrc")
    produtos = navegador.find_elements(By.CLASS_NAME, "i0X6df")
    
    melhor_preco = pmax
    melhor_nome = melhor_link = ""
    for p in produtos:
        texto = p.text
        textos = texto.split("\n")                    
        
        nome = ancora = ""
        preco = 0
        for t in textos:
            print(f"texto é: {t}")
            if "R$" in t:
                if "Entrega" in t:
                    pass
                else:
                    preco = t
                    padrao = re.compile(r"(R\$\s?\w+\.\w{3},\w{2})")
                    strings = re.findall(padrao, preco)
                    print(strings)
                    preco = strings[0]
                    preco = preco.replace("R$ ", "").replace(".", "").replace(",", ".")

                for alg in preco:
                    if alg == ".":
                        pass
                    elif alg.isnumeric():
                        pass
                    else:
                        preco = preco.replace(alg, "")

            if "Visitar o site de" in t:
                ancora = t
                print(f"ancora é: {ancora}")

            if produto.lower() in t.lower():
                nome = t
                print(f"nome é: {nome}")
        
        
        termos_banidos_google = banido.split(" ")
        termo_banido_google = False
        
        for termo in termos_banidos_google:
            if termo in nome:
                termo_banido_google = True
        
        if nome == "" or ancora == "":
            print("não passou. falta algo")
            pass 
        
        elif preco == 0:
            pass
        
        elif termo_banido_google == True:
            print("não passou. banido está nele")
            pass
        
        else:
        
            try:    
                link = navegador.find_element(By.PARTIAL_LINK_TEXT, f"{ancora}").get_attribute("href")
                print(f"link é: {link}")
            except:
                print("não passou, link não existe")
                pass

            preco = float(preco)

            if preco < pmin:
                print("não passou, preço menor que o min")
                pass
            elif preco > pmax:
                print("não passou, preço maior que o max")
                pass
            else:
                print(f"dicionário é: {dict_temp}")
                if preco < melhor_preco:
                    melhor_preco = preco
                    melhor_nome = nome
                    melhor_link = link
                    print(f"melhor preco é: {melhor_preco}")
                    print(f"melhor nome é: {melhor_nome}")
                    print(f"melhor link é: {melhor_link}")
                else:
                    pass
    
    print(melhor_nome, melhor_preco, melhor_link)
    print(f"dicionário é: {dict_temp}")
    dict_temp = {"Produto": melhor_nome, "Preço": melhor_preco, "Link de Acesso": melhor_link}

    lista_final.append(dict_temp.copy())
    dict_temp.clear()
    
    # ir no buscapé


navegador.close()
print(lista_final)
    
    
    
    
    
    
    
'''lista_produtos = navegador.find_elements(By.CLASS_NAME, "C7Lkve")
    print(lista_produtos)
    

    melhor_link = ""
    for oferta in lista_produtos:
        texto = oferta.text # pega todos os textos da div separados por \n (nome, preço e vendedor)
        textos = texto.split("\n")
        print(textos)

        # verificar se palavra banida está no nome do produto
        if banido in textos[0]:
            pass
        
        # verificar se nome to produto está de acordo com o da planilha
        elif produto.lower() not in textos[0].lower():
            pass
        
        # se passar dos checks anteriores, tratar preço
        else:
            preco = textos[1].replace("R$ ", "").replace(".", "").replace(",", ".")
            for alg in preco:
                if alg.isalpha():
                    preco = preco.replace(f"{alg}", "")

            
            if float(preco) < pmax:
                melhor_preco = float(preco)
                melhor_nome = textos[0]
                print(melhor_nome)
                print(melhor_preco)
                navegador.find_element(By.PARTIAL_LINK_TEXT, textos[0]).click()'''
                
            

texto é: iPhone 11 64 GB - Branco Apple
texto é: 4,6
texto é: 52.259
texto é: 4,6 de 5 estrelas. 52.259 resenhas de produto.
texto é: Smartphone · 1 chip · 4G · iOS
texto é: R$ 3.215,02
['R$ 3.215,02']
texto é: R$ 3.215,02.
['R$ 3.215,02']
texto é: Americanas.com
texto é: Frete não incluído
texto é: . Visitar o site de Americanas.com em uma nova janela
ancora é: . Visitar o site de Americanas.com em uma nova janela
texto é: Comparar preços de 5 ou mais lojas
não passou. falta algo
texto é: iPhone 12 64GB - Roxo - Estou Zerado
nome é: iPhone 12 64GB - Roxo - Estou Zerado
texto é: Smartphone · 1 chip · 5G · iOS
texto é: R$ 3.212,06
['R$ 3.212,06']
texto é: R$ 3.212,06.
['R$ 3.212,06']
texto é: Recondicionado
texto é: Trocafy
texto é: Entrega gratuita
texto é: . Visitar o site de Trocafy em uma nova janela
ancora é: . Visitar o site de Trocafy em uma nova janela
link é: https://www.google.com.br/url?url=https://www.trocafy.com.br/iphone-12-64gb-roxo-sou-como-novo-2555/p%3Fidsku%3D979%26sr

link é: https://www.google.com.br/url?url=https://www.trocafy.com.br/iphone-12-64gb-roxo-sou-como-novo-2555/p%3Fidsku%3D979%26srsltid%3DAfAwrE445Wq1WPRl1-UKQrrIBPrWUtEj8oV4XQIi1PmuGOx4HQXuk9BrQ_U&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwj66s_Sion-AhUbHbkGHeAiCsMQguUECIkJ&usg=AOvVaw3S-dWfm5JmqL4JhmkqXEU3
dicionário é: {}
texto é: iPhone 12 Mini 256GB - Verde - Estou Zerado
texto é: 4,6
texto é: 1.968
texto é: 4,6 de 5 estrelas. 1.968 resenhas de produto.
texto é: Smartphone · 1 chip · 5G · iOS
texto é: R$ 3.259,31
['R$ 3.259,31']
texto é: R$ 3.259,31.
['R$ 3.259,31']
texto é: Recondicionado
texto é: Trocafy
texto é: Entrega gratuita
texto é: . Visitar o site de Trocafy em uma nova janela
ancora é: . Visitar o site de Trocafy em uma nova janela
texto é: Comparar preços de 2 lojas
não passou. falta algo
texto é: Xiaomi 12 Pro Dual Sim 12 GB + 256 GB Verde / GENUINE / 24 Meses de Garantia Promo
texto é: 4,4
texto é: 321
texto é: 4,4 de 5 estrelas. 321 resenhas de produto.
texto é: 2 chips · Tela OL

link é: https://www.google.com.br/url?url=https://produto.mercadolivre.com.br/MLB-3232407781-iphone-11-64-gb-preto-vitrine-apple-com-garantia-nf-_JM%3Fmatt_tool%3D18956390%26utm_source%3Dgoogle_shopping%26utm_medium%3Dorganic&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwj66s_Sion-AhUbHbkGHeAiCsMQgOUECLAK&usg=AOvVaw06N52hPAVqh2lBHRYrZOvJ
dicionário é: {}
texto é: iPhone 11 Pro 64 GB Dourado
texto é: 4,6
texto é: 8.975
texto é: 4,6 de 5 estrelas. 8.975 resenhas de produto.
texto é: Smartphone · 2 chips · 4G · iOS
texto é: R$ 3.499,99
['R$ 3.499,99']
texto é: R$ 3.499,99.
['R$ 3.499,99']
texto é: Mercado Livre
texto é: Entrega gratuita
texto é: . Visitar o site de Mercado Livre em uma nova janela
ancora é: . Visitar o site de Mercado Livre em uma nova janela
não passou. falta algo
texto é: Apple Watch Series 7 45mm Preto Gps Tela Retina Oled Chip S7
texto é: 4,7
texto é: 6.041
texto é: 4,7 de 5 estrelas. 6.041 resenhas de produto.
texto é: Oversized · De pulso · Sono
texto é: R$ 3.499,99
['R$ 3.499,99

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.111)
Stacktrace:
Backtrace:
	(No symbol) [0x0077DCE3]
	(No symbol) [0x007139D1]
	(No symbol) [0x00624DA8]
	(No symbol) [0x0060D0D3]
	(No symbol) [0x0066EA8B]
	(No symbol) [0x0067D093]
	(No symbol) [0x0066ACC6]
	(No symbol) [0x00646F68]
	(No symbol) [0x006480CD]
	GetHandleVerifier [0x009F3832+2506274]
	GetHandleVerifier [0x00A29794+2727300]
	GetHandleVerifier [0x00A2E36C+2746716]
	GetHandleVerifier [0x00826690+617600]
	(No symbol) [0x0071C712]
	(No symbol) [0x00721FF8]
	(No symbol) [0x007220DB]
	(No symbol) [0x0072C63B]
	BaseThreadInitThunk [0x76857D69+25]
	RtlInitializeExceptionChain [0x7751B74B+107]
	RtlClearBits [0x7751B6CF+191]


In [216]:
# ir no buscapé
from selenium.webdriver.support.select import Select


navegador = webdriver.Chrome() # retirar
navegador.get(caminhos[1])

for busca in df_buscas.index:
    
    navegador.get(caminhos[1])
    
    # esperar página carregar
    elemento = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="new-header"]/div[1]')))
    
    # retirar
    produto = df_buscas.loc[busca, "Nome"]
    banido = df_buscas.loc[busca, "Termos banidos"]
    pmin = df_buscas.loc[busca, "Preço mínimo"]
    pmax = df_buscas.loc[busca, "Preço máximo"]
    
    # buscar produto eliminando termo banido
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(f"{produto}", Keys.ENTER)
    
    # aguardar
    elemento = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'RefinementGroup_Group__i_fAT')))
    
    # filtrar relevantes
    navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/select').click()
    sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/select/option[3]').click()
    navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/select').send_keys(Keys.ENTER)
    
    # Pegar produtos
    disponiveis = navegador.find_elements(By.CLASS_NAME, "SearchCard_ProductCard_Inner__7JhKb")
    
    
    
    
    
    
    melhor_preco = pmax
    melhor_nome = melhor_link = ""
    for disponivel in disponiveis:
        texto = disponivel.text
        textos = texto.split("\n")
        print(textos)
        
        nome = ancora = ""
        preco = 0
        for t in textos:
            print(f"texto é: {t}")
            if "R$" in t:
                if "entrega" in t.lower() or "frete" in t.lower() or "juros" in t.lower():
                    pass
                else:
                    preco = t
                    padrao = re.compile(r"(R\$\s?\w+\.\w{3},\w{2})")
                    strings = re.findall(padrao, preco)
                    print(strings)
                    preco = strings[0]
                    preco = preco.replace("R$ ", "").replace(".", "").replace(",", ".")

                for alg in preco:
                    if alg == ".":
                        pass
                    elif alg.isnumeric():
                        pass
                    else:
                        preco = preco.replace(alg, "")

            if produto.lower() in t.lower():
                nome = t
                print(f"nome é: {nome}")
        
        if nome == "":
            print("não passou. falta algo")
            pass 
        
        elif preco == 0:
            pass
        
        elif banido in nome:
            print("não passou. banido está nele")
            pass
        
        else:
        
            try:    
                link = disponivel.get_attribute("href")
                print(f"link é: {link}")
            except:
                print("não passou, link não existe")
                pass

            preco = float(preco)

            if preco < pmin:
                print("não passou, preço menor que o min")
                pass
            elif preco > pmax:
                print("não passou, preço maior que o max")
                pass
            else:
                print(f"dicionário é: {dict_temp}")
                if preco < melhor_preco:
                    melhor_preco = preco
                    melhor_nome = nome
                    melhor_link = link
                    print(f"melhor preco é: {melhor_preco}")
                    print(f"melhor nome é: {melhor_nome}")
                    print(f"melhor link é: {melhor_link}")
                else:
                    pass
    
    print(melhor_nome, melhor_preco, melhor_link)
    print(f"dicionário é: {dict_temp}")
    dict_temp = {"Produto": melhor_nome, "Preço": melhor_preco, "Link de Acesso": melhor_link}

    lista_final.append(dict_temp.copy())
    dict_temp.clear()
    
    
    
    
    #lista_select = Select(lista)
    #lista_select.select_by_visible_text("Menor preço")
    
    #navegador.find_element(By.ID, "orderBy").click()
    #sleep(1)
    #navegador.find_element(By.XPATH, '//*[@id="orderBy"]/option[2]').click()
    
    
    '''# filtrar por preços
    elementos_preco = navegador.find_elements(By.CLASS_NAME, 'Input_Input__93_dl')
    for campo in elementos_preco:
        valor = campo.get_attribute("value")
        for _ in range(len(valor)):
            campo.click()
            campo.send_keys(Keys.DELETE)
            print(campo.get_attribute("value"))
            
        if campo == elementos_preco[0]:
            campo.click()
            campo.send_keys(Keys.DELETE)
            campo.send_keys(f"{pmin * 100}")
        else:
            campo.click()
            campo.send_keys(Keys.DELETE)
            campo.send_keys(f"{pmax * 100}")
    valor = navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[1]/div/aside/div[2]/div/div[2]/div[1]/div/input').get_attribute("value")
    print(f" o valor final é {valor}")
    #navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[1]/div/aside/div[2]/div/div[2]/div[1]/div/input').send_keys(Keys.DELETE) #,f"{pmin}")
    
    #navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[1]/div/aside/div[2]/div/div[2]/div[2]/div/input').clear()
    #navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[1]/div/aside/div[2]/div/div[2]/div[2]/div/input').send_keys(f"{pmax}")
    
    #navegador.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div/div[1]/div/aside/div[2]/div/div[2]/button').click()'''


['8º', 'Smartphone Apple iPhone 12 64GB Câmera Dupla', '4.8 (4209)', 'Menor preço via Mercado Livre', 'R$ 3.599,99', 'até 10x de R$ 360,00 sem juros', 'Frete Grátis', '0.5% de volta', 'Compare entre 15 lojas']
texto é: 8º
texto é: Smartphone Apple iPhone 12 64GB Câmera Dupla
nome é: Smartphone Apple iPhone 12 64GB Câmera Dupla
texto é: 4.8 (4209)
texto é: Menor preço via Mercado Livre
texto é: R$ 3.599,99
['R$ 3.599,99']
texto é: até 10x de R$ 360,00 sem juros
texto é: Frete Grátis
texto é: 0.5% de volta
texto é: Compare entre 15 lojas
link é: https://www.buscape.com.br/celular/smartphone-apple-iphone-12-64gb-ios?_lc=88&searchterm=iphone%2012%2064gb
não passou, preço maior que o max
['Smartphone Apple iPhone 12 Mini 64GB Câmera Dupla', '4.7 (97)', 'Menor preço via Fast Shop', 'R$ 4.583,05', 'até 6x de R$ 812,40 com juros', '0.5% de volta', 'Compare entre 5 lojas']
texto é: Smartphone Apple iPhone 12 Mini 64GB Câmera Dupla
texto é: 4.7 (97)
texto é: Menor preço via Fast Shop
texto é: R$

link é: https://www.buscape.com.br/lead?oid=937934447&channel=86&index=6&searchterm=rtx%203060
não passou, preço menor que o min
['Placa de Video PNY NVIDIA Geforce RTX 3060, 12 GB GDDR6, DLSS, Ray Tracing - VCG306012DFBPB1', 'Menor preço via KaBuM!', 'R$ 2.299,99', 'até 10x de R$ 270,58 com juros', '0.5% de volta', 'KaBuM']
texto é: Placa de Video PNY NVIDIA Geforce RTX 3060, 12 GB GDDR6, DLSS, Ray Tracing - VCG306012DFBPB1
nome é: Placa de Video PNY NVIDIA Geforce RTX 3060, 12 GB GDDR6, DLSS, Ray Tracing - VCG306012DFBPB1
texto é: Menor preço via KaBuM!
texto é: R$ 2.299,99
['R$ 2.299,99']
texto é: até 10x de R$ 270,58 com juros
texto é: 0.5% de volta
texto é: KaBuM
link é: https://www.buscape.com.br/lead?oid=920801032&channel=86&index=7&searchterm=rtx%203060
não passou, preço menor que o min
['Placa de Vídeo RTX 3060 Asus Dual O12G V2 NVIDIA GeForce, 12GB GDDR6, LHR, DLSS, Ray Tracing - DUAL-RTX3060-O12G-V2', 'Menor preço via KaBuM!', 'R$ 2.499,99', 'até 10x de R$ 294,11 com juros',

In [240]:
# junção

import pandas as pd
import re
import win32com.client as win32

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from time import sleep


# ler base de dados
df_buscas = pd.read_excel("buscas.xlsx")


# armazenar caminho dos sites de pesquisa
caminhos = ["https://shopping.google.com.br/?pli=1", "https://www.buscape.com.br/?og=19220&og=19220&gclid=Cj0KCQjwiZqhBhCJARIsACHHEH_J8GkuCKXw8zsWebQLekAbLNQvl2k2uQRMtR-KTUyu5MFUXUY7PL8aAr6sEALw_wcB"]

# abrir navegador
navegador = webdriver.Chrome()


lista_consolidada = []
lista_final_buscape = []
lista_final_google = []


# pesquisar na barra de busca cada produto
for busca in df_buscas.index:
    
    # pegar primeiro elemento (google)
    navegador.get(caminhos[0])

    # esperar página carregar
    elemento_espera = WebDriverWait(navegador, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/c-wiz[3]/div/div/div[1]/div/div/div')))
    
    produto = df_buscas.loc[busca, "Nome"]
    banido = df_buscas.loc[busca, "Termos banidos"]
    pmin = df_buscas.loc[busca, "Preço mínimo"]
    pmax = df_buscas.loc[busca, "Preço máximo"]
    
    # buscar produto eliminando termo banido (-termo_que não quero)
    navegador.find_element(By.ID, "REsRA").send_keys(f"{produto} -{banido}", Keys.ENTER)
    
    # aguardar
    elemento_espera = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'sh-dr__restricts')))
    
    # filtrar por preços
    navegador.find_element(By.NAME, "lower").send_keys(f"{pmin}")
    navegador.find_element(By.NAME, "upper").send_keys(f"{pmax}", Keys.ENTER)
    
    # aparecer do menor para o maior preço
    navegador.find_element(By.CLASS_NAME, "Yf5aUd").click()
    sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="lb"]/div/g-menu/g-menu-item[3]').click()
    
    
    # termos procurados
    termos_procurados = produto.split(" ")
    

    
    # guardar produtos em uma lista
    disponiveis_google = navegador.find_elements(By.CLASS_NAME, "i0X6df")
    
    melhor_preco_google = pmax
    melhor_nome_google = melhor_link_google = ""
    
    for disponivel_google in disponiveis_google:
        texto_google = disponivel_google.text
        textos_google = texto_google.split("\n")                    
        
        
        print(f" o que estamos buscando: {termos_procurados}")
        
        nome = ancora = ""
        preco = 0
        
        for t_google in textos_google:
            print(f"texto é {t_google}")
            if "R$" in t_google:
                if "entrega" in t_google.lower():
                    print("identificado como entrega")
                    pass
                else:
                    print(f"preço inicial é: {t_google}")
                    preco_inicial = t_google
                    padrao = re.compile(r"(R\$\s?\w+\.\w{3},\w{2})")
                    strings = re.findall(padrao, preco_inicial)
                    preco = strings[0]
                    preco = preco.replace("R$ ", "").replace(".", "").replace(",", ".")
                    print(f"preço final é: {preco}")
                '''for alg in preco:
                    if alg == ".":
                        pass
                    elif alg.isnumeric():
                        pass
                    else:
                        preco = preco.replace(alg, "")'''
            
            termo_encontrado_google = True
            
            for termo_procurado in termos_procurados:
                print(f"termo que estamos procurando: {termo_procurado}")
                if termo_procurado.lower() not in t_google.lower():
                    print(f"termo '{termo_procurado.lower()}' não foi encontrado em '{t_google.lower()}'")
                    termo_encontrado_google = False
            
            if termo_encontrado_google is True:
                print(f"reconhecido como nome: {t_google}")
                nome = t_google
            
            '''if produto.lower() in t.lower():
                print("reconhecido como nome: {t_google}")
                nome = t_google'''
                

            if "Visitar o site de" in t_google:
                print(f"reconhecido como link: {t_google}")
                ancora = t_google

                
        termos_banidos_google = banido.split(" ")
        print(f"termos banidos: {termos_banidos_google}")
        termo_banido_google = False
        
        for termo in termos_banidos_google:
            if termo.lower() in nome.lower():
                print("reconheceu um termo banido no nome")
                termo_banido_google = True
        
        
        if nome == "" or ancora == "":
            print("descartado por não possuir nome ou link")
            pass 
        
        elif preco == 0:
            print("descartado por não possuir preço")
            pass
        
        elif termo_banido_google is True:
            print("descartado por possuir termo banido")
            pass
        
        
        else:
        
            try:    
                link = navegador.find_element(By.PARTIAL_LINK_TEXT, f"{ancora}").get_attribute("href")
            except:
                print("descartado por não possuir link de ligação")
                pass

            preco = float(preco)

            if preco < pmin:
                print("descartado por ser inferior ao preço minimo")
                pass
            elif preco > pmax:
                print("descartado por ser superior ao preço máximo")
                pass
            else:
                if preco < melhor_preco_google:
                    print("considerado até então a melhor opção")
                    melhor_preco_google = preco
                    melhor_nome_google = nome
                    melhor_link_google = link
                else:
                    print("descartado por não ser a melhor opção até então")
                    pass
    
    dict_temp_google = {"Produto": melhor_nome_google, "Solicitado": produto, "Preço": melhor_preco_google, "Link de Acesso": melhor_link_google}
    
    lista_final_google.append(dict_temp_google.copy())
    dict_temp_google.clear()
    
    
    
    # ir no buscapé
    navegador.get(caminhos[1])

    # esperar página carregar
    elemento_espera = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="new-header"]/div[1]')))
    
    # buscar produto eliminando termo banido
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(f"{produto}", Keys.ENTER)
    
    # aguardar
    elemento_espera = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'RefinementGroup_Group__i_fAT')))
    
    # filtrar relevantes
    navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/select').click()
    sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/select/option[3]').click()
    navegador.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/select').send_keys(Keys.ENTER)
    
    # Pegar produtos
    disponiveis_buscape = navegador.find_elements(By.CLASS_NAME, "SearchCard_ProductCard_Inner__7JhKb")
    
    melhor_preco_buscape = pmax
    melhor_nome_buscape = melhor_link_buscape = ""
    
    for disponivel_buscape in disponiveis_buscape:
        texto_buscape = disponivel_buscape.text
        textos_buscape = texto_buscape.split("\n")
        
        
        nome = ancora = ""
        preco = 0
        for t_buscape in textos_buscape:
            if "R$" in t_buscape:
                if "entrega" in t_buscape.lower() or "frete" in t_buscape.lower() or "juros" in t_buscape.lower():
                    pass
                else:
                    preco_inicial = t_buscape
                    padrao = re.compile(r"(R\$\s?\w+\.\w{3},\w{2})")
                    strings = re.findall(padrao, preco_inicial)
                    preco = strings[0]
                    preco = preco.replace("R$ ", "").replace(".", "").replace(",", ".")

                '''for alg in preco:
                    if alg == ".":
                        pass
                    elif alg.isnumeric():
                        pass
                    else:
                        preco = preco.replace(alg, "")'''
            
            termo_encontrado_buscape = True
        
            for termo_procurado in termos_procurados:
                if termo_procurado.lower() not in nome.lower():
                    termo_encontrado_buscape = False
            
            if termo_encontrado_buscape is True:
                nome = t_buscape
            
        
        termos_banidos_buscape = banido.split(" ")
        termo_banido_buscape = False
        
        
        for termo in termos_banidos_buscape:
            if termo.lower() in nome.lower():
                termo_banido_buscape = True
                

        
        if nome == "":
            pass 
        
        elif preco == 0:
            pass
        
        elif termo_banido_buscape is True:
            pass
        
        
        else:
        
            try:    
                link = disponivel.get_attribute("href")
            except:
                pass

            preco = float(preco)

            if preco < pmin:
                pass
            elif preco > pmax:
                pass
            else:
                if preco < melhor_preco_buscape:
                    melhor_preco_buscape = preco
                    melhor_nome_buscape = nome
                    melhor_link_buscape = link

                else:
                    pass
                
    dict_temp_buscape = {"Produto": melhor_nome_buscape, "Soliciatado": produto, "Preço": melhor_preco_buscape, "Link de Acesso": melhor_link_buscape}

    lista_final_buscape.append(dict_temp_buscape.copy())
    dict_temp_buscape.clear()
    

    
navegador.close()


# tratamento final
for ind, prod in enumerate(df_buscas["Nome"]):
    
    preco_google = lista_final_google[ind]["Preço"]
    preco_buscape = lista_final_buscape[ind]["Preço"]
    
    produto_google = lista_final_google[ind]["Produto"]
    produto_buscape = lista_final_buscape[ind]["Produto"]
    
    if produto_google == "" and produto_buscape == "":
        lista_consolidada.append({"Produto": "Não encontrado", "Solicitado": prod, "Preço": "-", "Link de Acesso": "-"})
    elif produto_google == "":
        lista_consolidada.append(lista_final_buscape[ind])
    elif produto_buscape == "":
        lista_consolidada.append(lista_final_google[ind])
    else:
        if preco_google > preco_buscape:
            lista_consolidada.append(lista_final_buscape[ind])
        else:
            lista_consolidada.append(lista_final_google[ind])
    

df_consolidada = pd.DataFrame(lista_consolidada)

df_consolidada.to_excel("compras.xlsx", index=False)

# Abrir outlook/ criar uma instância do outlook
outlook = win32.Dispatch("outlook.application")
    
# Criar email
mail = outlook.CreateItem(0)

# Escrever destinatário
mail.To = "ric.accorsi@gmail.com"

# Escrever assunto
mail.Subject = "Levantamento de Preços"

# Escrever corpo do texto
mail.Body = '''
Olá, Fulano!

Tudo bem! Segue em anexo o resultado da busca acerca dos melhores preços para os produtos solicitados.
Qualquer dúvida, me coloco à disposição!

Att,

Ricardo
'''

anexo = r"C:\Users\ricac\Curso Python Impressionador - Hashtag Treinamentos\Projeto 2 - Automação Web\compras.xlsx"

# Adicionar anexo ao email
mail.Attachments.Add(anexo)

# Enviar email
mail.Send()

print("Programa concluído com sucesso")

 o que estamos buscando: ['iphone', '12', '64gb']
texto é iPhone 12 64GB - Roxo - Estou Zerado
termo que estamos procurando: iphone
termo que estamos procurando: 12
termo que estamos procurando: 64gb
reconhecido como nome: iPhone 12 64GB - Roxo - Estou Zerado
texto é R$ 3.059,10
preço inicial é: R$ 3.059,10
preço final é: 3059.10
termo que estamos procurando: iphone
termo 'iphone' não foi encontrado em 'r$ 3.059,10'
termo que estamos procurando: 12
termo '12' não foi encontrado em 'r$ 3.059,10'
termo que estamos procurando: 64gb
termo '64gb' não foi encontrado em 'r$ 3.059,10'
texto é R$ 3.059,10.
preço inicial é: R$ 3.059,10.
preço final é: 3059.10
termo que estamos procurando: iphone
termo 'iphone' não foi encontrado em 'r$ 3.059,10.'
termo que estamos procurando: 12
termo '12' não foi encontrado em 'r$ 3.059,10.'
termo que estamos procurando: 64gb
termo '64gb' não foi encontrado em 'r$ 3.059,10.'
texto é Recondicionado
termo que estamos procurando: iphone
termo 'iphone' não foi enco

descartado por não ser a melhor opção até então
 o que estamos buscando: ['iphone', '12', '64gb']
texto é (Seminovo) iPhone 12 Apple Azul, 64GB
termo que estamos procurando: iphone
termo que estamos procurando: 12
termo que estamos procurando: 64gb
reconhecido como nome: (Seminovo) iPhone 12 Apple Azul, 64GB
texto é R$ 3.144,15
preço inicial é: R$ 3.144,15
preço final é: 3144.15
termo que estamos procurando: iphone
termo 'iphone' não foi encontrado em 'r$ 3.144,15'
termo que estamos procurando: 12
termo '12' não foi encontrado em 'r$ 3.144,15'
termo que estamos procurando: 64gb
termo '64gb' não foi encontrado em 'r$ 3.144,15'
texto é R$ 3.144,15.
preço inicial é: R$ 3.144,15.
preço final é: 3144.15
termo que estamos procurando: iphone
termo 'iphone' não foi encontrado em 'r$ 3.144,15.'
termo que estamos procurando: 12
termo '12' não foi encontrado em 'r$ 3.144,15.'
termo que estamos procurando: 64gb
termo '64gb' não foi encontrado em 'r$ 3.144,15.'
texto é taQi
termo que estamos procur

 o que estamos buscando: ['iphone', '12', '64gb']
texto é iPhone 12 Seminovo
termo que estamos procurando: iphone
termo que estamos procurando: 12
termo que estamos procurando: 64gb
termo '64gb' não foi encontrado em 'iphone 12 seminovo'
texto é R$ 3.499,00
preço inicial é: R$ 3.499,00
preço final é: 3499.00
termo que estamos procurando: iphone
termo 'iphone' não foi encontrado em 'r$ 3.499,00'
termo que estamos procurando: 12
termo '12' não foi encontrado em 'r$ 3.499,00'
termo que estamos procurando: 64gb
termo '64gb' não foi encontrado em 'r$ 3.499,00'
texto é R$ 3.499,00.
preço inicial é: R$ 3.499,00.
preço final é: 3499.00
termo que estamos procurando: iphone
termo 'iphone' não foi encontrado em 'r$ 3.499,00.'
termo que estamos procurando: 12
termo '12' não foi encontrado em 'r$ 3.499,00.'
termo que estamos procurando: 64gb
termo '64gb' não foi encontrado em 'r$ 3.499,00.'
texto é Usado
termo que estamos procurando: iphone
termo 'iphone' não foi encontrado em 'usado'
termo que est

considerado até então a melhor opção
 o que estamos buscando: ['rtx', '3060']
texto é Placa De Vídeo Galax Geforce RTX 3060 1 Click Oc 12GB GDDR6
termo que estamos procurando: rtx
termo que estamos procurando: 3060
reconhecido como nome: Placa De Vídeo Galax Geforce RTX 3060 1 Click Oc 12GB GDDR6
texto é 4,4
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '4,4'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '4,4'
texto é 19
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '19'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '19'
texto é 4,4 de 5 estrelas. 19 resenhas de produto.
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '4,4 de 5 estrelas. 19 resenhas de produto.'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '4,4 de 5 estrelas. 19 resenhas de produto.'
texto é PCI Express · HDMI · GDDR
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em 

descartado por não ser a melhor opção até então
 o que estamos buscando: ['rtx', '3060']
texto é Placa De Vídeo Galax Geforce RTX 2060 Oc 6Gb 26NRL7HPX7OC
termo que estamos procurando: rtx
termo que estamos procurando: 3060
termo '3060' não foi encontrado em 'placa de vídeo galax geforce rtx 2060 oc 6gb 26nrl7hpx7oc'
texto é 4,7
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '4,7'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '4,7'
texto é 68
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '68'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '68'
texto é 4,7 de 5 estrelas. 68 resenhas de produto.
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '4,7 de 5 estrelas. 68 resenhas de produto.'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '4,7 de 5 estrelas. 68 resenhas de produto.'
texto é PCI Express · DVI · HDMI · GDDR
termo que estamos procurando: rtx
termo '

descartado por não ser a melhor opção até então
 o que estamos buscando: ['rtx', '3060']
texto é Zotac Geforce RTX 3060 Ti Gaming Amp White Edition LHR Graphics Card, Chipset ...
termo que estamos procurando: rtx
termo que estamos procurando: 3060
reconhecido como nome: Zotac Geforce RTX 3060 Ti Gaming Amp White Edition LHR Graphics Card, Chipset ...
texto é 4,4
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '4,4'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '4,4'
texto é 15
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '15'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '15'
texto é 4,4 de 5 estrelas. 15 resenhas de produto.
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em '4,4 de 5 estrelas. 15 resenhas de produto.'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em '4,4 de 5 estrelas. 15 resenhas de produto.'
texto é PCI Express · HDMI · ZOTAC
termo que es

 o que estamos buscando: ['rtx', '3060']
texto é Placa de Vídeo Galax GeForce RTX 3060 TI 1-Click OC Plus LHR 8GB GDDR6X 256 bits ...
termo que estamos procurando: rtx
termo que estamos procurando: 3060
reconhecido como nome: Placa de Vídeo Galax GeForce RTX 3060 TI 1-Click OC Plus LHR 8GB GDDR6X 256 bits ...
texto é PCI Express · HDMI · GDDR
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em 'pci express · hdmi · gddr'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em 'pci express · hdmi · gddr'
texto é R$ 4.299,90
preço inicial é: R$ 4.299,90
preço final é: 4299.90
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em 'r$ 4.299,90'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em 'r$ 4.299,90'
texto é R$ 4.299,90.
preço inicial é: R$ 4.299,90.
preço final é: 4299.90
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em 'r$ 4.299,90.'
termo que estamos procurando: 3060
termo '3060' não foi encontrado

descartado por não ser a melhor opção até então
 o que estamos buscando: ['rtx', '3060']
texto é Emtek Geforce Black Edition Rtx 3080 Oc 10gb
termo que estamos procurando: rtx
termo que estamos procurando: 3060
termo '3060' não foi encontrado em 'emtek geforce black edition rtx 3080 oc 10gb'
texto é PCI Express · GDDR
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em 'pci express · gddr'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em 'pci express · gddr'
texto é R$ 4.500,00
preço inicial é: R$ 4.500,00
preço final é: 4500.00
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em 'r$ 4.500,00'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em 'r$ 4.500,00'
texto é R$ 4.500,00.
preço inicial é: R$ 4.500,00.
preço final é: 4500.00
termo que estamos procurando: rtx
termo 'rtx' não foi encontrado em 'r$ 4.500,00.'
termo que estamos procurando: 3060
termo '3060' não foi encontrado em 'r$ 4.500,00.'
texto é Mercado Livr

In [241]:
df_resultados = pd.read_excel("compras.xlsx")
display(df_resultados)
for link in df_resultados["Link de Acesso"]:
    print(link)

,Produto,Solicitado,Preço,Link de Acesso
0,iPhone 12 64GB - Roxo - Estou Zerado,iphone 12 64gb,3059.10,https://www.google.com.br/url?url=https://www....
1,Placa De Vídeo Geforce RTX 3060 V2 Dual 12GB G...,rtx 3060,4016.11,https://www.google.com.br/url?url=https://www....


https://www.google.com.br/url?url=https://www.trocafy.com.br/iphone-12-64gb-roxo-sou-como-novo-2555/p%3Fidsku%3D979%26srsltid%3DAfAwrE76_nWJpDcjgSOF0wpwfVlSeaVjIgUlAkyJum5dBjCRBLRH_NSv3yA&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwi_quyToo7-AhVXqpUCHYAQAtwQguUECO0E&usg=AOvVaw0VsVX2TV6yAkaFSncX3qfY
https://www.google.com.br/url?url=https://www.americanas.com.br/produto/4278760371%3Fopn%3DYSMESP%26offerId%3D61854053d9fd6edeec6919f8%26srsltid%3DAfAwrE5E8iR-Ik0Fqjd9-3SZTDA0gCMQAT5VFN4dXeVzuU_xGmJf6sMff7M&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjzp6Ocoo7-AhUXkZUCHTfqCxQQguUECNsG&usg=AOvVaw3IeropG042U4l13_RZZAeA
